# *추가데이터

# date_week_v2

In [ ]:
# 날짜-요일 테이블 만들기
import calendar

def weekday(ymd):
    week =['Mon','Tue','Wed','Thr','Fri','Sat','Sun']
    year, mon,day = int(ymd[0:4]), int(ymd[4:6]), int(ymd[6:8]) 
    idx = datetime.date(year,mon,day).weekday()
    return week[idx], idx

calendar.monthrange(2020,1)

date_df = pd.DataFrame([], columns=['base_ymd','mon','weekday','week_num'])


idx = 0
for year in range(2018,2021):
    for mon in range(1,13):
        _, limit_day = calendar.monthrange(year,mon)
        for day in range(1,limit_day+1):
            ymd = '{0:04d}{1:02d}{2:02d}'.format(year,mon,day)
            wd,wn = weekday(ymd)
            date_df.loc[idx] = [int(ymd),mon,wd,wn] 
            idx += 1

# date_df.to_csv("date_week_v2.csv",index=False)

# * BigQuery 전처리

# 1. fpopl_v2

###### table 구성예시 (총 47546443 행)
필드 이름	유형	모드	
base_ymd	INTEGER	NULLABLE	
tmzon_se_code	INTEGER	NULLABLE	
sexdstn_se_code	STRING	NULLABLE	
agrde_se_code	STRING	NULLABLE	
popltn_cascnt	INTEGER	NULLABLE	
adstrd_code_str	STRING	NULLABLE	
code	STRING	NULLABLE	
adstrd_nm	STRING	NULLABLE	
brtc_nm	STRING	NULLABLE	
signgu_nm	STRING	NULLABLE	
mon	INTEGER	NULLABLE	
weekday	STRING	NULLABLE	
week_num	INTEGER	NULLABLE	

In [7]:
'''
SELECT * EXCEPT(adstrd_code, base_ymd_str)
FROM 
  (SELECT * 
  FROM(
    SELECT *, SUBSTR(adstrd_code_str,0,5) AS code
    FROM(
      SELECT *, CAST(adstrd_code AS STRING) AS adstrd_code_str, CAST(base_ymd AS STRING) AS base_ymd_str
      FROM KT_data.fpopl)
      ) AS A JOIN KT_data.adstrd_master AS B USING(adstrd_code)
    ) AS A2 JOIN ADD_data.date_week_v2 AS B2 USING(base_ymd)
ORDER BY base_ymd, tmzon_se_code, agrde_se_code, sexdstn_se_code, adstrd_code_str;
'''

'\nSELECT * EXCEPT(adstrd_code, base_ymd_str)\nFROM \n  (SELECT * \n  FROM(\n    SELECT *, SUBSTR(adstrd_code_str,0,5) AS code\n    FROM(\n      SELECT *, CAST(adstrd_code AS STRING) AS adstrd_code_str, CAST(base_ymd AS STRING) AS base_ymd_str\n      FROM KT_data.fpopl)\n      ) AS A JOIN KT_data.adstrd_master AS B USING(adstrd_code)\n    ) AS A2 JOIN ADD_data.date_week_v2 AS B2 USING(base_ymd)\nORDER BY base_ymd, tmzon_se_code, agrde_se_code, sexdstn_se_code, adstrd_code_str;\n'

# 2. 1_week9allmean_v2

* 구성예시   
{
    "code": "11110",  
    "weekday": "Mon",  
    "week_num": "0",  
    "signgu_nm": "종로구",  
    "Mean_CNT": "154.3989"  
  },

In [ ]:
'''
SELECT code, weekday, week_num,  signgu_nm, ROUND(AVG(popltn_cascnt),4) AS Mean_CNT
FROM PROCESS_data.fpopl_v2
GROUP BY weekday,week_num, signgu_nm, code
ORDER BY code, week_num
'''

# 3. 2_tzallmean

* 구성예시  
{
    "tmzon_se_code": "0",  
    "code": "11110",  
    "signgu_nm": "종로구",  
    "Mean_CNT": "70.66537"  
  },

In [ ]:
'''
SELECT tmzon_se_code, code, signgu_nm, ROUND(AVG(popltn_cascnt),5) AS Mean_CNT
FROM PROCESS_data.fpopl_v2
GROUP BY tmzon_se_code, code, signgu_nm
ORDER BY tmzon_se_code, code
'''

# 4. delivery_day9cnt

In [1]:
'''
SELECT PROCESS_DT, code, DLVR_STORE_INDUTY_NM, COUNT(DLVR_STORE_INDUTY_NM) AS CNT
FROM(
  SELECT *, SUBSTR(STORECODE_STR,0,5) AS code
  FROM(
    SELECT *, CAST(DLVR_STORE_LEGALDONG_CODE AS STRING) AS STORECODE_STR
    FROM KT_data.delivery
    WHERE DLVR_STORE_SIDO ='서울특별시'))
GROUP BY PROCESS_DT, code, DLVR_STORE_INDUTY_NM
ORDER BY PROCESS_DT
'''

"\nSELECT PROCESS_DT, code, DLVR_STORE_INDUTY_NM, COUNT(DLVR_STORE_INDUTY_NM) AS CNT\nFROM(\n  SELECT *, SUBSTR(STORECODE_STR,0,5) AS code\n  FROM(\n    SELECT *, CAST(DLVR_STORE_LEGALDONG_CODE AS STRING) AS STORECODE_STR\n    FROM KT_data.delivery\n    WHERE DLVR_STORE_SIDO ='서울특별시'))\nGROUP BY PROCESS_DT, code, DLVR_STORE_INDUTY_NM\nORDER BY PROCESS_DT\n"